In [38]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [39]:
df=pd.read_csv('raw.csv',header=None,index_col=False)

In [40]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,40.766,-73.978,40.768,-73.955,1.60,4.12,1.0,187.0,1.0,0.0,33.0,187.0,5.0,0.0,177.0,186.0
1,1.0,40.767,-73.971,40.741,-73.989,2.00,5.27,1.0,187.0,4.0,0.0,9.0,187.0,9.0,0.0,184.0,91.0
2,2.0,40.790,-73.955,40.791,-73.949,1.31,8.00,1.0,187.0,7.0,0.0,5.0,187.0,15.0,0.0,177.0,259.0
3,3.0,40.766,-73.976,40.788,-73.976,1.80,7.06,1.0,187.0,9.0,0.0,34.0,187.0,16.0,0.0,177.0,164.0
4,4.0,40.781,-73.961,40.812,-73.938,2.90,9.00,1.0,187.0,14.0,0.0,13.0,187.0,23.0,0.0,203.0,275.0


In [55]:
tazCount=318

In [56]:
OD=np.zeros((tazCount,tazCount))

In [57]:
for i in range(len(df)):
    pid=df.iloc[i,15]-1
    aid=df.iloc[i,16]-1
    if not np.isnan(pid+aid):
        OD[int(pid)][int(aid)]+=1

In [91]:
def CalcSinglyConstrained(ProdA, AttrA, F):
    '''Calculates singly constrained trip distribution for a given friction factor matrix
    ProdA = Production array
    AttrA = Attraction array
    F = Friction factor matrix
    Resutrns trip table
    '''
    SumAjFij = (AttrA*F).sum(1)
    SumAjFij[SumAjFij==0]=0.0001
    return ProdA*(AttrA*F).transpose()/SumAjFij

In [123]:
eps=1e-5

In [171]:
Pro=np.sum(OD, axis=0)+eps
Att=Attadj=np.sum(OD, axis=1)+eps
F=OD.transpose()
predictedOD=CalcSinglyConstrained(Pro, Att, F)
MAE=np.mean(abs(np.array(predictedOD)-np.array(OD)))
print (MAE)
for i in range(10):
    Pro=np.sum(predictedOD, axis=0)+eps
    Att=np.sum(predictedOD, axis=1)+eps
    Attadj=Attadj*(np.sum(OD, axis=1)/Att)
    F=OD.transpose()
    predictedOD=CalcSinglyConstrained(Pro, Attadj, F)
    MAE=np.mean(abs(np.array(predictedOD)-np.array(OD)))
    print (MAE)

2.8010850525016147
0.2062104035199805
0.058170072295067134
0.016815888937201443
0.004841216081296985
0.0013902230907991702
0.00039879881904892494
0.00011441022362327492
3.287978041957989e-05
9.515795875060941e-06
2.8551143185055923e-06


In [183]:
Pro=Proadj=np.sum(OD, axis=0)
Att=Attadj=np.sum(OD, axis=1)
F=OD.transpose()
predictedOD=CalcDoublyConstrained(Pro, Att, F, maxIter = 10)
MAE=np.mean(abs(np.array(predictedOD)-np.array(OD)))
print (MAE)

Checking production, attraction balancing:
Production:  518721.0
Attraction:  518721.0
Production, attraction balancing OK.
3.984989060978653


In [131]:
def CalcDoublyConstrained(ProdA, AttrA, F, maxIter = 10):
    '''Calculates doubly constrained trip distribution for a given friction factor matrix
    ProdA = Production array
    AttrA = Attraction array
    F = Friction factor matrix
    maxIter (optional) = maximum iterations, default is 10
    Returns trip table
    '''
    Trips1 = np.zeros((len(ProdA),len(ProdA)))
    print ('Checking production, attraction balancing:')
    sumP = sum(ProdA)
    sumA = sum(AttrA)
    print ('Production: ', sumP)
    print ('Attraction: ', sumA)
    if sumP != sumA:
        print ('Productions and attractions do not balance, attractions will be scaled to productions!')
        AttrA = AttrA*(sumP/sumA)
        AttrT = AttrA.copy()
        ProdT = ProdA.copy()
    else:
        print ('Production, attraction balancing OK.')
        AttrT = AttrA.copy()
        ProdT = ProdA.copy()

    for balIter in range(0, maxIter):
        for i in range(0,len(ProdA)):
            Trips1[i,:] = ProdA[i]*AttrA*F[i,:]/max(0.000001, sum(AttrA * F[i,:]))
        
        #Run 2D balancing --->
        ComputedAttractions = Trips1.sum(0) 
        ComputedAttractions[ComputedAttractions==0]=1
        AttrA = AttrA*(AttrT/ComputedAttractions)

        ComputedProductions = Trips1.sum(1)
        ComputedProductions[ComputedProductions==0]=1
        ProdA = ProdA*(ProdT/ComputedProductions)

    for i in range(0,len(ProdA)):
            Trips1[i,:] = ProdA[i]*AttrA*F[i,:]/max(0.000001, sum(AttrA * F[i,:]))
            
    return Trips1